In [ ]:
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0


In [ ]:
!nvidia-smi

Sat Jun 22 20:10:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install torch

In [ ]:
!pip3 install torchvision

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision
import numpy as np
from torchvision import transforms
import os

In [ ]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
DOWNLOAD_ROOT = os.path.join(os.pardir,"MNIST_data")

train1_dataset = datasets.MNIST(root=DOWNLOAD_ROOT,train=True,transform=transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()]),download=True)
test1_dataset = datasets.MNIST(root=DOWNLOAD_ROOT,train=False,transform=transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()]),download=True)

In [ ]:
print("train data 개수 :",train1_dataset.data.shape[0])
print("test data 개수 :",test1_dataset.data.shape[0])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_dataset=train1_dataset
test_dataset=test1_dataset

In [ ]:
split_ratio=0
train_dataset,_=train_test_split(train1_dataset,test_size=1-split_ratio,random_state=7)
test_dataset,_=train_test_split(test1_dataset,test_size=1-split_ratio,random_state=7)
train_nosplit_dataset,_=train_test_split(train1_dataset,test_size=1,random_state=7)
test_nosplit_dataset,_=train_test_split(test1_dataset,test_size=1,random_state=7)

In [ ]:
print("train data 개수 :",len(train_dataset))
print("test data 개수 :",len(test_dataset))
print("nosplit train data 개수 :",len(train_nosplit_dataset))
print("nosplit test data 개수 :",len(test_nosplit_dataset))

In [ ]:
class LeNet_5(nn.Module):
    def __init__(self):
        super(LeNet_5,self).__init__()
        self.c1 = nn.Conv2d(1,6,5)
        self.avgpool1 = nn.AvgPool2d(2)
        self.c2 = nn.Conv2d(6,16,5)
        self.avgpool2 = nn.AvgPool2d(2)
        self.c3 = nn.Conv2d(16,120,5)
        self.n1 = nn.Linear(120,84)
        self.sig=nn.Sigmoid()
        self.n2 = nn.Linear(84,10)

    def forward(self,x):
        x = self.c1(x)
        x = self.avgpool1(x)
        x = self.c2(x)
        x = self.avgpool2(x)
        x = self.c3(x)
        x = torch.flatten(x,1)
        x = self.n1(x)
        x = self.sig(x)
        x = self.n2(x)
        return x

In [ ]:
class LeNet_improving(nn.Module):
    def __init__(self):
        super(LeNet_improving,self).__init__()
        self.c1 = nn.Conv2d(1,6,5)
        self.maxpool1 = nn.MaxPool2d(2)
        self.c2 = nn.Conv2d(6,16,5)
        self.maxpool2 = nn.MaxPool2d(2)
        self.c3 = nn.Conv2d(16,120,5)
        self.n1 = nn.Linear(120,84)
        self.relu=nn.ReLU()
        self.n2 = nn.Linear(84,10)

    def forward(self,x):
        x = self.c1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.c2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.c3(x)
        x = self.relu(x)
        x = torch.flatten(x,1)
        x = self.n1(x)
        x = self.relu(x)
        x = self.n2(x)
        return x

In [ ]:
#default
LOAD = True
SEED=7
torch.manual_seed(SEED)
if device=='cuda':
  torch.cuda.manual_seed_all(SEED)
BATCH_SIZE = 128
EPOCHS = 80
LEARNING_RATE = 0.1
loss_function = nn.CrossEntropyLoss()
model = LeNet_5()
model.zero_grad()
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)


loss_list = []
accuracy_list = []
epoch = 0

In [ ]:
train_dataset_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
test_dataset_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
train_nosplit_dataset_loader = DataLoader(train_nosplit_dataset, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)
test_nosplit_dataset_loader = DataLoader(test_nosplit_dataset, batch_size=BATCH_SIZE, shuffle=True,drop_last=True)

In [ ]:
TOTAL_BATCH = len(train_dataset_loader)
print(TOTAL_BATCH)

In [ ]:
PATH = os.path.join('/content',"pretrained_model.pt")
SAVE_INTERVAL = 5

In [ ]:
while epoch < EPOCHS:
    cost = 0
    for image, label in train_dataset_loader:
        optimizer.zero_grad()
        predicted = model.forward(image)
        loss = loss_function(predicted,label)
        loss.backward()
        optimizer.step()
        cost+=loss
    with torch.no_grad():
        total = 0
        correct = 0
        for image, label in test_dataset_loader:
            out = model(image)
            _,predict = torch.max(out.data, 1)
            total += label.size(0)
            correct += (predict==label).sum()
    average_cost = cost/TOTAL_BATCH
    accuracy = 100*correct/total
    loss_list.append(average_cost.detach().numpy())
    accuracy_list.append(accuracy)
    epoch+=1
    print("epoch : {} | loss : {:.6f}" .format(epoch, average_cost))
    print("Accuracy : {:.2f}".format(accuracy))
    print("---------------------")
    if epoch%5 ==0:
        torch.save({"epoch":epoch,"loss list":loss_list,"accuracy list":accuracy_list,"model":model.state_dict(),"optimizer":optimizer.state_dict()},PATH)

In [ ]:
#improving
PATH = os.path.join('/content',"pretrained_model2.pt")
LOAD = True
BATCH_SIZE = 128
EPOCHS = 80
LEARNING_RATE = 0.1
loss_function = nn.CrossEntropyLoss()
model = LeNet_improving()
model.zero_grad()
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)


loss_list2 = []
accuracy_list2 = []
epoch = 0

In [ ]:
while epoch < EPOCHS:
    cost = 0
    for image, label in train_dataset_loader:
        optimizer.zero_grad()
        predicted = model.forward(image)
        loss = loss_function(predicted,label)
        loss.backward()
        optimizer.step()
        cost+=loss
    with torch.no_grad():
        total = 0
        correct = 0
        for image, label in test_dataset_loader:
            out = model(image)
            _,predict = torch.max(out.data, 1)
            total += label.size(0)
            correct += (predict==label).sum()
    average_cost = cost/TOTAL_BATCH
    accuracy = 100*correct/total
    loss_list2.append(average_cost.detach().numpy())
    accuracy_list2.append(accuracy)
    epoch+=1
    print("epoch : {} | loss : {:.6f}" .format(epoch, average_cost))
    print("Accuracy : {:.2f}".format(accuracy))
    print("---------------------")
    if epoch%5 ==0:
        torch.save({"epoch":epoch,"loss list":loss_list2,"accuracy list":accuracy_list2,"model":model.state_dict(),"optimizer":optimizer.state_dict()},PATH)

In [ ]:
TOTAL_BATCH = len(train_nosplit_dataset_loader)
print(TOTAL_BATCH)

468


In [ ]:
#learning late 0.001
PATH = os.path.join('/content',"pretrained_model3.pt")
LOAD = True
BATCH_SIZE = 128
EPOCHS = 80
LEARNING_RATE = 0.0001
loss_function = nn.CrossEntropyLoss()
model = LeNet_5()
model.zero_grad()
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)


loss_list3 = []
accuracy_list3 = []
epoch = 0

In [ ]:
while epoch < EPOCHS:
    cost = 0
    for image, label in train_nosplit_dataset_loader:
        optimizer.zero_grad()
        predicted = model.forward(image)
        loss = loss_function(predicted,label)
        loss.backward()
        optimizer.step()
        cost+=loss
    with torch.no_grad():
        total = 0
        correct = 0
        for image, label in test_nosplit_dataset_loader:
            out = model(image)
            _,predict = torch.max(out.data, 1)
            total += label.size(0)
            correct += (predict==label).sum()
    average_cost = cost/TOTAL_BATCH
    accuracy = 100*correct/total
    loss_list3.append(average_cost.detach().numpy())
    accuracy_list3.append(accuracy)
    epoch+=1
    print("epoch : {} | loss : {:.6f}" .format(epoch, average_cost))
    print("Accuracy : {:.2f}".format(accuracy))
    print("---------------------")
    if epoch%5 ==0:
        torch.save({"epoch":epoch,"loss list":loss_list3,"accuracy list":accuracy_list3,"model":model.state_dict(),"optimizer":optimizer.state_dict()},PATH)

In [ ]:
#nosplit, batch 256
PATH = os.path.join('/content',"pretrained_model4.pt")
LOAD = True
BATCH_SIZE = 256
EPOCHS = 80
LEARNING_RATE = 0.01
loss_function = nn.CrossEntropyLoss()
model = LeNet_5()
model.zero_grad()
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)


loss_list4 = []
accuracy_list4 = []
epoch = 0

In [ ]:
while epoch < EPOCHS:
    cost = 0
    for image, label in train_nosplit_dataset_loader:
        optimizer.zero_grad()
        predicted = model.forward(image)
        loss = loss_function(predicted,label)
        loss.backward()
        optimizer.step()
        cost+=loss
    with torch.no_grad():
        total = 0
        correct = 0
        for image, label in test_nosplit_dataset_loader:
            out = model(image)
            _,predict = torch.max(out.data, 1)
            total += label.size(0)
            correct += (predict==label).sum()
    average_cost = cost/TOTAL_BATCH
    accuracy = 100*correct/total
    loss_list4.append(average_cost.detach().numpy())
    accuracy_list4.append(accuracy)
    epoch+=1
    print("epoch : {} | loss : {:.6f}" .format(epoch, average_cost))
    print("Accuracy : {:.2f}".format(accuracy))
    print("---------------------")
    if epoch%5 ==0:
        torch.save({"epoch":epoch,"loss list":loss_list4,"accuracy list":accuracy_list4,"model":model.state_dict(),"optimizer":optimizer.state_dict()},PATH)

In [ ]:

if LOAD:
    check = torch.load(PATH)
    model.load_state_dict(check["model"])
    epoch = check["epoch"]
    accuracy_list = check["accuracy list"]
    loss_list = check["loss list"]
    optimizer.load_state_dict(check["optimizer"])

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(np.arange(0,EPOCHS),loss_list)
plt.plot(np.arange(0,EPOCHS),loss_list2)
plt.plot(np.arange(0,EPOCHS),loss_list3)
plt.plot(np.arange(0,EPOCHS),loss_list4)
plt.legend(['1','2','3','4'])
plt.subplot(1,2,2)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(np.arange(0,EPOCHS), accuracy_list)
plt.plot(np.arange(0,EPOCHS), accuracy_list2)
plt.plot(np.arange(0,EPOCHS), accuracy_list3)
plt.plot(np.arange(0,EPOCHS), accuracy_list4)
plt.legend(['1','2','3','4'])
plt.show()